<a href="https://colab.research.google.com/github/BooBoa/Stable_diffusion-dreamlikediffusion/blob/main/Stable_Diffusion_2_1_Dreamlike_art_dreamlike_diffusion_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Run This Cell first: will install all requirements.
!pip install "ipywidgets>=7,<8"
%pip install --quiet --upgrade diffusers transformers scipy mediapy accelerate ftfy spacy
import subprocess
# The xformers package is mandatory to be able to create several 768x768 images.
github_url = "https://github.com/brian6091/xformers-wheels"
xformer_id = "0.0.15.dev0+4c06c79"
xformers_wheels = f"xformers-{xformer_id}.d20221205-cp38-cp38-linux_x86_64.whl"
# Install xformers using pre-compiled Python wheels
%pip install -q {github_url}/releases/download/{xformer_id}/{xformers_wheels}

In [ ]:

#@title Dreamlike Diffusion
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display
from numpy import int16

model_id = "dreamlike-art/dreamlike-diffusion-1.0"           
scheduler = DDIMScheduler(beta_start=0.00085,
                          beta_end=0.012,
                          beta_schedule="scaled_linear",
                          clip_sample=False,
                          set_alpha_to_one=False)

pipe = StableDiffusionPipeline.from_pretrained(model_id,
                                               scheduler=scheduler,
                                               safety_checker=None,
                                               torch_dtype=torch.float16).to("cuda")

pipe.enable_attention_slicing()
pipe.enable_xformers_memory_efficient_attention()


g_cuda = None
g_cuda = torch.Generator(device='cuda')
seed = 65743829 #@param {type:"number"}
g_cuda.manual_seed(seed)
prompt = "dreamlikeart apocalyptic person and there pet, a breathtaking digital illustration, beautiful anime characters, wearing hazmat suits, post apocalytic on background" #@param {type:"string"}
negative_prompt = "bright eyes, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, blurry, deformed, ugly, bad anatomy, bad proportions, extra limbs, cloned face, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck" #@param {type:"string"}
num_samples = 100 #@param {type:"slider", min:1, max:100, step:1}
guidance_scale = 9.5 #@param {type:"slider", min:1, max:20, step:0.1}
num_inference_steps = 30 #@param {type:"slider", min:1, max:500, step:1}
height = 768 #@param {type:"number"}
width = 768 #@param {type:"number"}

for i in range(num_samples):
  with autocast("cuda"), torch.inference_mode():
      images = pipe(
          prompt,
          height=height,
          width=width,
          negative_prompt=negative_prompt,
          num_inference_steps=num_inference_steps,
          guidance_scale=guidance_scale,
          generator=g_cuda
      ).images
  seed += 100

  for img in images:
      img.save(f"Seed_{seed}.png")  
      display(img)

In [ ]:
#@title Stable-diffusion-2-1
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import torch
from torch import autocast
from IPython.display import display


# model_id = 'stabilityai/stable-diffusion-2'
model_id = 'stabilityai/stable-diffusion-2-1'

scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler")

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    revision="fp16",
    torch_dtype=torch.float16,
    scheduler=scheduler
).to("cuda")

pipe.enable_attention_slicing()
pipe.enable_xformers_memory_efficient_attention()

g_cuda = None
g_cuda = torch.Generator(device="cuda")
seed = 64738 #@param{type:"number"}
g_cuda.manual_seed(seed)
prompt = "a post apocalyptic girl, a breathtaking digital illustration, beautiful anime characters, wearing hazmat suits, post apocalytic on background, "#@param{type:"string"}
neg_prompt = " low res, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, blurry, deformed, ugly, bad anatomy, bad proportions, extra limbs, cloned face, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"#@param{type:"string"}
num_samples = 10 #@param{type:"slider", min:1, max:100, steps:1}
steps = 35 #@param{type:"slider", min:10, max:500, steps:1}
guidance_scale = 9.5 #@param{type:"slider", min:1, max:20, step:0.1}
width = 768 #@param{type:"number"}
height = 768 #@param{type:"number"}

for i in range(num_samples):
    with autocast("cuda"):
        images = pipe(
            prompt = prompt,
            negative_prompt = neg_prompt,
            num_inference_steps = steps,
            guidance_scale = guidance_scale,
            width = width,
            height = height,
            generator = g_cuda
        ).images

    seed += 100

    for img in images:
        img.save(f"seed_{seed}.png")
        display(img)